**ФИО:** Смольков Максим Дмитриевич

**Группа:** 25152

# **Сборка генома на основе Графа де Брейна:**

Входные данные:

Последовательность нуклеотидов вида:

```
sequence = "ACCGGTACA"
```

и размер k-мера:

```
k = 3
```

Необходимо реализовать:
1. Функцию, которая формирует все k-меры последовательности нуклеотидов и возвращает список ребер графа и словарь, в котором ключами выступают вершины графа, а значеняими - входящие и исходящие степени вершин;

2. Функцию, принимающую список ребер графа и словарь с входящими и исходящими степенями вершин графа, которая восстанавливает исходную последовательность, используя алгоритм поиска эйлерова пути.


In [1]:
from collections import defaultdict

def get_k_mers_and_graph(sequence, k):
    """
    Формирует k-меры и строит граф де Брейна
    
    Возвращает:
    - edges: список ребер (k-меры)
    - vertex_dict: словарь вершин с их степенями
    """
    
    # Получаем все k-меры из последовательности
    edges = []
    for i in range(len(sequence) - k + 1):
        k_mer = sequence[i:i + k]
        edges.append(k_mer)
    
    # Создаем вершины графа (k-1)-меры
    vertices = set()
    for edge in edges:
        vertices.add(edge[:-1])  # префикс (k-1) символов
        vertices.add(edge[1:])   # суффикс (k-1) символов
    
    # Инициализируем словарь для хранения степеней вершин
    vertex_dict = {}
    for vertex in vertices:
        vertex_dict[vertex] = {"in": 0, "out": 0}
    
    # Заполняем степени вершин
    for edge in edges:
        prefix = edge[:-1]
        suffix = edge[1:]
        
        # Увеличиваем исходящую степень префикса
        vertex_dict[prefix]["out"] += 1
        
        # Увеличиваем входящую степень суффикса
        vertex_dict[suffix]["in"] += 1
    
    return edges, vertex_dict


def find_eulerian_path(edges, vertex_dict):
    """
    Находит эйлеров путь в графе де Брейна и восстанавливает последовательность
    
    Возвращает:
    - Собранную последовательность
    """
    
    # Строим список смежности для графа
    adjacency_list = defaultdict(list)
    for edge in edges:
        prefix = edge[:-1]
        suffix = edge[1:]
        adjacency_list[prefix].append(suffix)
    
    # Находим начальную вершину (с out > in)
    start_vertex = None
    end_vertex = None
    
    for vertex, degrees in vertex_dict.items():
        in_deg = degrees["in"]
        out_deg = degrees["out"]
        
        if out_deg - in_deg == 1:
            start_vertex = vertex
        elif in_deg - out_deg == 1:
            end_vertex = vertex
    
    # Если не нашли явную начальную вершину, берем первую
    if start_vertex is None:
        start_vertex = list(vertex_dict.keys())[0]
    
    # Используем алгоритм Флери для поиска эйлерова пути
    path = []
    stack = [start_vertex]
    
    # Копируем список смежности для удаления пройденных ребер
    temp_adj = defaultdict(list)
    for key, values in adjacency_list.items():
        temp_adj[key] = values.copy()
    
    while stack:
        current = stack[-1]
        
        if temp_adj[current]:
            # Есть непройденные ребра
            next_vertex = temp_adj[current].pop(0)
            stack.append(next_vertex)
        else:
            # Нет непройденных ребер - добавляем в путь
            path.append(stack.pop())
    
    # Переворачиваем путь (алгоритм возвращает в обратном порядке)
    path = path[::-1]
    
    # Восстанавливаем последовательность из пути
    if not path:
        return ""
    
    # Первая вершина - это начало последовательности
    assembled_sequence = path[0]
    
    # Добавляем последний символ каждой следующей вершины
    for vertex in path[1:]:
        assembled_sequence += vertex[-1]
    
    return assembled_sequence


def assemble_genome(sequence, k):
    """
    Основная функция для сборки генома
    """
    print(f"Исходная последовательность: {sequence}")
    print(f"Длина k-мера: {k}")
    print("-" * 50)
    
    # 1. Получаем k-меры и граф
    edges, vertex_dict = get_k_mers_and_graph(sequence, k)
    
    print(f"Все k-меры (ребра графа):")
    for i, edge in enumerate(edges):
        print(f"  {i+1}. {edge}")
    
    print(f"\nВершины графа (k-1)-меры и их степени:")
    for vertex, degrees in vertex_dict.items():
        print(f"  {vertex}: входящая={degrees['in']}, исходящая={degrees['out']}")
    
    print("-" * 50)
    
    # 2. Восстанавливаем последовательность
    assembled = find_eulerian_path(edges, vertex_dict)
    
    print(f"Собранная последовательность: {assembled}")
    print(f"Исходная последовательность: {sequence}")
    print(f"Совпадают: {assembled == sequence}")
    
    return assembled

In [2]:
sequence = "ACCGGTACA"
k = 3
result = assemble_genome(sequence, k)

Исходная последовательность: ACCGGTACA
Длина k-мера: 3
--------------------------------------------------
Все k-меры (ребра графа):
  1. ACC
  2. CCG
  3. CGG
  4. GGT
  5. GTA
  6. TAC
  7. ACA

Вершины графа (k-1)-меры и их степени:
  CC: входящая=1, исходящая=1
  GG: входящая=1, исходящая=1
  CA: входящая=1, исходящая=0
  TA: входящая=1, исходящая=1
  CG: входящая=1, исходящая=1
  AC: входящая=1, исходящая=2
  GT: входящая=1, исходящая=1
--------------------------------------------------
Собранная последовательность: ACCGGTACA
Исходная последовательность: ACCGGTACA
Совпадают: True


In [6]:
result

'ACCGGTACA'